##Goal:  Train a Naive Bayes model to classify future SMS messages as either spam or ham.

Steps:

1.  Convert the words ham and spam to a binary indicator variable(0/1)

2.  Convert the txt to a sparse matrix of TFIDF vectors

3.  Fit a Naive Bayes Classifier

4.  Measure your success using roc_auc_score



In [76]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score

In [77]:
df= pd.read_csv("SMSSpamCollection",sep='\t', names=['spam', 'txt'])

In [78]:
df.head()

,spam,txt
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [79]:
#TFIDF Vectorizer, just like before
stopset = set(stopwords.words('english'))
vectorizer = TfidfVectorizer(use_idf=True, lowercase=True, strip_accents='ascii', stop_words=stopset)

In [80]:
#need to transfrom "spam" and "ham" into 0's and 1's
dummies = pd.get_dummies(df.spam)
# Update df to include dummy and drop the char variable
df = pd.concat([df.txt, dummies.spam], axis=1)

In [81]:
df.head()

,txt,spam
0,"Go until jurong point, crazy.. Available only ...",0.0
1,Ok lar... Joking wif u oni...,0.0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1.0
3,U dun say so early hor... U c already then say...,0.0
4,"Nah I don't think he goes to usf, he lives aro...",0.0


In [82]:
#in this case our dependent variable will be spam as 1 and ham as 0
y = df.spam

In [83]:
#convert df.txt from text to features
X= vectorizer.fit_transform(df.txt)

In [84]:
#5572 observations x 8587 unique words.
print y.shape
print X.shape

(5572L,)
(5572, 8587)


In [85]:
#Test Train Split as usual
X_train, X_test,y_train, y_test = train_test_split(X, y, random_state=42)

In [86]:
#we will train a naive_bayes classifier
clf = naive_bayes.MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [87]:
#We can test our model's accuracy like this:

roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])

0.98589322144123448

In [88]:
#spam detector [1 = positive for spam)]
spam_test_array=np.array(["Buy now!! Limited Time Offer! Free Direct 100% Legal Lion Tiger Zebra exotic rare meats Order Direct Click here ==> http://ExoticMeatsMarket.com free shipping"])

spam_test_vector = vectorizer.transform(spam_test_array)

print clf.predict(spam_test_vector)

[ 1.]


In [89]:
#ham detector [0 = not spam]
ham_test_array=np.array(["See you at 10, my flight was delayed for 2 hours.  Leaving Laguardia at 8pm now"])

ham_test_vector = vectorizer.transform(ham_test_array)

print clf.predict(ham_test_vector)

[ 0.]
